In [1]:
import pennylane as qml
from pennylane import numpy as np
from numpy.linalg import qr

In [2]:
def qr_haar(N):
    '''Generate a Haar-random matrix using the QR decomposition'''
    
    A, B = np.random.normal(size=(N, N)), np.random.normal(size=(N, N))
    Z = A + 1j * B
    Q, R = qr(Z)
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(N)])
    return np.dot(Q, Lambda)

In [3]:
dev = qml.device("default.qubit", wires = 1)

In [5]:
mat = qr_haar(2)
op = qml.transforms.zyz_decomposition(mat, 0)
print(op)

[Rot(tensor(-2.40800534, requires_grad=True), tensor(2.66345697, requires_grad=True), tensor(1.32433778, requires_grad=True), wires=[0])]


In [6]:
params = op[0].parameters
params = np.array([params[0], params[1], params[2]], requires_grad=True)

In [9]:
@qml.qnode(dev)
def haar_circuit(params):
    qml.Rot(*params, wires=0)
    return qml.expval(qml.PauliX(0))

In [10]:
opt = qml.GradientDescentOptimizer(0.1)

for _ in range(3):
    params, loss = opt.step_and_cost(haar_circuit, params)
    print(params, loss)

[-2.40800534  2.68511804  1.36895987] 0.11225712018159106
[-2.40800534  2.70311237  1.41214372] 0.08836394531549696
[-2.40800534  2.71741656  1.45406691] 0.06707596865414722
